In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import SAGEConv, GATConv, Linear, HGTConv, HeteroConv, GCNConv, to_hetero
from tqdm import tqdm
from tor

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

2.0.1


In [2]:
dataset = OGB_MAG(root='./data', preprocess='metapath2vec')
data = dataset[0]

print(data)

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 5416271] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] }
)


In [3]:
paper_node_data = data['paper']
cites_edge_data = data['paper', 'cites', 'paper']

In [4]:
dataset = OGB_MAG(root='./data', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


model = GNN(hidden_channels=64, out_channels=dataset.num_classes)
model = to_hetero(model, data.metadata(), aggr='sum')

In [7]:
from torch_geometric.nn import GATConv, Linear, to_hetero

class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATConv((-1, -1), hidden_channels, add_self_loops=False)
        self.lin1 = Linear(-1, hidden_channels)
        self.conv2 = GATConv((-1, -1), out_channels, add_self_loops=False)
        self.lin2 = Linear(-1, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) + self.lin1(x)
        x = x.relu()
        x = self.conv2(x, edge_index) + self.lin2(x)
        return x


model = GAT(hidden_channels=64, out_channels=dataset.num_classes)
model = to_hetero(model, data.metadata(), aggr='sum')

In [8]:
train_loader = NeighborLoader(
    data,
    # Sample 15 neighbors for each node and each edge type for 2 iterations:
    num_neighbors=[15] * 2,
    # Use a batch size of 128 for sampling training nodes of type "paper":
    batch_size=64,
    input_nodes=('paper', data['paper'].train_mask),
)

In [9]:
print(next(iter(train_loader)))

HeteroData(
  paper={
    x=[11357, 128],
    year=[11357],
    y=[11357],
    train_mask=[11357],
    val_mask=[11357],
    test_mask=[11357],
    n_id=[11357],
    input_id=[64],
    batch_size=64
  },
  author={
    x=[2375, 128],
    n_id=[2375]
  },
  institution={
    x=[177, 128],
    n_id=[177]
  },
  field_of_study={
    x=[1637, 128],
    n_id=[1637]
  },
  (author, affiliated_with, institution)={
    edge_index=[2, 0],
    e_id=[0]
  },
  (author, writes, paper)={
    edge_index=[2, 3163],
    e_id=[3163]
  },
  (paper, cites, paper)={
    edge_index=[2, 5877],
    e_id=[5877]
  },
  (paper, has_topic, field_of_study)={
    edge_index=[2, 6003],
    e_id=[6003]
  },
  (institution, rev_affiliated_with, author)={
    edge_index=[2, 414],
    e_id=[414]
  },
  (paper, rev_writes, author)={
    edge_index=[2, 2988],
    e_id=[2988]
  },
  (field_of_study, rev_has_topic, paper)={
    edge_index=[2, 5272],
    e_id=[5272]
  }
)


In [33]:
def train():
    for batch in tqdm(train_loader):
        model.train()
        optimizer.zero_grad()
        out = model(batch.x_dict, batch.edge_index_dict)
        mask = batch['paper'].train_mask
        loss = torch.nn.functional.cross_entropy(out['paper'][mask], batch['paper'].y[mask])
        loss.backward()
        optimizer.step()
    return float(loss)

In [34]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)  # Define optimizer.
loss = train()

  2%|▏         | 230/9838 [00:29<20:38,  7.76it/s]


KeyboardInterrupt: 